# working with Larger Databases
### Refer to the database schema while studying this notebook
![Schema](schema.png)

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('chinook.db')

In [3]:
pd.read_sql_query("select * from sqlite_master where type='table';",conn)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...



# Joining Three Tables
Write a query that gathers data about the invoice with an invoice_id of 4. Include the following columns in order:
- The **id of the track**, track_id.
- The **name of the track**, track_name.
- The **name of media type of the track**, track_type.
- The **price that the customer paid for the track**, unit_price.
- The **quantity of the track that was purchased**, quantity.
  
merge the 3 tables: **invoice_line, track, and media_type**


In [4]:
q1 = '''
SELECT 
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
where il.invoice_id = 4
'''
pd.read_sql_query(q1,conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


# Joining More Than Three Tables
Add a column containing the artists name to the previous query.
- The column should be called **artist_name**
- The column should be placed between **track_name** and **track_type**

merge the 5 tables: **invoice_line, track, media_type, album and artist**


In [5]:
q2 = '''
SELECT 
    il.track_id,
    t.name track_name,
    ar.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
where il.invoice_id = 4
'''
pd.read_sql_query(q2,conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


# Combining Multiple Joins with Subqueries


### list the top 10 artists, calculated by the number of times a track by that artist has been purchased.

In [6]:
q3 = '''
SELECT 
    ta.name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                ar.name,
                t.track_id
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
            ) ta 
            ON il.track_id = ta.track_id
GROUP BY 1
ORDER BY 2 DESC ;
'''
pd.read_sql_query(q3,conn)


,artist,tracks_purchased
0,Queen,192
1,Jimi Hendrix,187
2,Nirvana,130
3,Red Hot Chili Peppers,130
4,Pearl Jam,129
5,AC/DC,124
6,Guns N' Roses,124
7,Foo Fighters,121
8,The Rolling Stones,117
9,Metallica,106


### list top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should return the following columns, in order:
- **album**, the title of the album
- **artist**, the artist who produced the album
- **tracks_purchased** the total number of tracks purchased from that album

In [7]:
q4 = '''
SELECT 
    ta.title album,
    ta.name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                al.title,
                ar.name,
                t.track_id
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
            ) ta 
            ON il.track_id = ta.track_id
GROUP BY 1
ORDER BY 3 DESC LIMIT 5;
'''
pd.read_sql_query(q4,conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


# Recursive Joins

Write a query that returns information about each employer and their supervisor. Your query should return the following columns, in order:
- **employee_name** - containing the first_name and last_name columns separated by a space, eg Luke Skywalker
- **employee_title** - the title of that employee
- **supervisor_name** - the first and last name of the person the employee reports to, in the same format as    **employee_name**
- **supervisor_title** - the title of the person the employee reports to
- The report should include employees even if they do not report to another employee.

In [8]:
q5 = '''
SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1 
LEFT JOIN employee e2 on e1.reports_to = e2.employee_id
'''
pd.read_sql_query(q5,conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


# Pattern Matching Using Like
Write a query that finds the contact details a customer with a first_name containing Belle from the database.
Your query should include the following columns, in order:
- **first_name**
- **last_name**
- **phone**

In [9]:
q6 = '''
SELECT
    first_name,
    last_name,
    phone
FROM customer 
WHERE first_name LIKE "%belle"
'''
pd.read_sql_query(q6,conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99



# Generating Columns With The Case Statement

- Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.
- Your query should include the following columns, in order:
    - **customer_name** - containing the first_name and last_name columns separated by a space, eg Luke Skywalker.
    - **number_of_purchases**, counting the number of purchases made by each customer.
    - **total_spent** - the sum of customers total purchases made by each customer.
    - **customer_category** - a column that categorizes the customer based on their total purchases. The column should contain the following values:
        - **small spender** - If the customer's total purchases are less than \$40.
        - **big spender** - If the customer's total purchases are greater than \$100.
        - **regular** - If the customer's total purchases are between \$40 and \$100 (inclusive).
- Order your results by the customer_name column.

In [10]:
q7 = '''
SELECT
   c.first_name || " " || c.last_name customer_name,
   COUNT(i.invoice_id) number_of_purchases,
   SUM(i.total) total_spent,
   CASE
       WHEN sum(i.total) < 40 THEN 'small spender'
       WHEN sum(i.total) > 100 THEN 'big spender'
       ELSE 'regular'
       END
       AS customer_category
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
GROUP BY 1 ORDER BY 1;
'''
pd.read_sql_query(q7,conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular


# The With Clause
- Create a query that shows summary data about the playlists in the Chinook database:
    - Use a WITH clause to create a named subquery with the following info:
        - The unique ID for the playlist.
        - The name of the playlist.
        - The name of each track from the playlist.
        - The length of the each track in seconds.
    - Your final table should have the following columns, in order:
        - **playlist_id** - the unique ID for the playlist.
        - **playlist_name** - The name of the playlist.
        - **number_of_tracks** - A count of the number of tracks in the playlist.
        - **length_seconds** - The sum of the length of the playlist in seconds.

In [11]:
q8 = '''
WITH playlist_info AS
    (
     SELECT
         p.playlist_id,
         p.name playlist_name,
         t.name track_name,
         (t.milliseconds / 1000) length_seconds
     FROM playlist p
     INNER JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     INNER JOIN track t ON t.track_id = pt.track_id
    )

SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2;
'''
pd.read_sql_query(q8,conn)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049
1,3,TV Shows,213,500987
2,5,90’s Music,1477,397970
3,8,Music,3290,876049
4,9,Music Videos,1,294
5,10,TV Shows,213,500987
6,11,Brazilian Music,39,9464
7,12,Classical,75,21736
8,13,Classical 101 - Deep Cuts,25,6742
9,14,Classical 101 - Next Steps,25,7565


# Creating Views
- Create a view called **customer_gt_90_dollars**:
    - The view should contain the columns from **customers**, in their original order.
    - The view should contain only customers who have purchased more than \$90 in tracks from the store.
- After the SQL query that creates the view, write a second query to display your newly created view: **SELECT \* FROM chinook.customer_gt_90_dollars**;.
    - Make sure you use a semicolon **(;)** to indicate the end of each query.

In [12]:
q9 = '''
CREATE VIEW customer_usa AS 
     SELECT * FROM customer
     WHERE country = "USA";
'''

q10 = '''
CREATE VIEW customer_gt_90_dollars AS
    SELECT 
        c.* 
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY c.customer_id 
    HAVING SUM(i.total) > 90;
'''

q11 = ' SELECT * FROM customer_gt_90_dollars'

pd.read_sql_query(q11,conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
6,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
7,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
8,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
9,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


# Combining Rows With Union
Use **UNION** to produce table of customers in the USA or have spent more than $90, using the **customer_usa** and **customer_gt_90_dollars** views:
- The result should contain the columns from customers, in their original order.

In [13]:
q12 = '''
SELECT * FROM customer_usa
UNION
Select * FROM customer_gt_90_dollars
'''
pd.read_sql_query(q12,conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
6,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
7,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
8,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
9,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


# Combining Rows using Intersect and Except
- Write a query that works out how many customers that are in the USA and have purchased more than \$90 are assigned to each sales support agent. For the purposes of this exercise, no two employees have the same name.
- Your result should have the following columns, in order:
    - **employee_name** - The first_name and last_name of the employee separated by a space, eg **Luke Skywalker**.
    - **customers_usa_gt_90** - The number of customer assigned to that employee that are both from the USA and have have purchased more than $90 worth of tracks.
- The result should include all employees with the title **"Sales Support Agent"**, but not employees with any other title.
- Order your results by the **employee_name column**.

In [14]:
q13 = '''
WITH customers_usa_gt_90 AS
    (
     SELECT * FROM customer_usa

     INTERSECT

     SELECT * FROM customer_gt_90_dollars
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;
'''
pd.read_sql_query(q13,conn)

,employee_name,customers_usa_gt_90
0,Jane Peacock,0
1,Margaret Park,2
2,Steve Johnson,2


# Multiple Named Subqueries
- Write a query that uses multiple named subqueries in a **WITH** clause to gather total sales data on customers from India:
    - The first named subquery should return all customers that are from India.
    - The second named subquery should calculate the sum total for every customer.
    - The main query should join the two named subqueries, resulting in the following final columns:
        - **customer_name** - The first_name and last_name of the customer, separated by a space, eg **Luke Skywalker**.
        - **total_purchases** - The total amount spent on purchases by that customer.

In [15]:
q14 = '''
WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
'''
pd.read_sql_query(q14,conn)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


# Each Country's Best Customer
- Create a query to find the customer from each country that has spent the most money at our store. Your query should return the following columns, in order:
    - **country** - The name of each country that we have a customer from.
    - **customer_name** - The first_name and last_name of the customer from that country with the most total purchases, separated by a space, eg Luke Skywalker.
    - **total_purchased** - The total dollar amount that customer has purchased.
- The rows should be order by the country column in alphabetical order.

In [16]:
q15 = '''
WITH customer_info AS
(
    SELECT 
        c.first_name || " " || c.last_name customer_name,
        c.country,
        sum(i.total) total_purchased
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY 1
)
SELECT 
    country,
    customer_name,
    MAX(total_purchased) total_purchased
FROM customer_info
GROUP BY 1 ORDER BY 1
'''
pd.read_sql_query(q15,conn)

,country,customer_name,total_purchased
0,Argentina,Diego Gutiérrez,39.60
1,Australia,Mark Taylor,81.18
2,Austria,Astrid Gruber,69.30
3,Belgium,Daan Peeters,60.39
4,Brazil,Luís Gonçalves,108.90
5,Canada,François Tremblay,99.99
6,Chile,Luis Rojas,97.02
7,Czech Republic,František Wichterlová,144.54
8,Denmark,Kara Nielsen,37.62
9,Finland,Terhi Hämäläinen,79.20
